In [1]:
import warnings
warnings.filterwarnings("ignore")

import logging
import pandas as pd
import os
import gzip

from datetime import datetime
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

In [2]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [3]:
####################################################
############# Setear segun cada maquina ############
os.chdir("C:/diego_tools/labo3/dataset")
####################################################

In [4]:
def error_rate(y, y_pred):
    dif_abs = sum(abs(y - y_pred))
    suma_real = sum(y)
    
    return round(100*dif_abs/suma_real,2)

In [5]:
def print_error_rate_total_y_cat(df_pred):
    error_rate_total = error_rate(df_pred.tn_real,df_pred.tn_pred)
    printmd("**Error Rate Total:** " + str(error_rate_total))

    for cat1_iter in df_pred.cat1.unique():
        df_pred_cat = df_pred[df_pred.cat1 == cat1_iter]
        error_rate_cat = error_rate(df_pred_cat.tn_real,df_pred_cat.tn_pred)
        printmd("**Error Rate por Categoría " + cat1_iter + "**: " + str(error_rate_cat))

In [6]:
df_sellout = pd.read_csv("emp3_sellout_base_period_product.csv")

In [7]:
df_sellout.head()

,periodo,periodo_fecha,product_id,tn,cust_request_qty,cust_request_tn,plan_precios_cuidados,cat1,cat2,cat3,sku_size,temp_media,temp_max_media,temp_min_media,IPC,promedio_mens_dolar_venta,catastrofe
0,201701,2017-01-01,20001,934.77222,479,937.72717,0,HC,ROPA LAVADO,Liquido,3000.0,25.65,30.4,20.9,1.6,15.91,False
1,201701,2017-01-01,20002,550.15707,391,555.18654,0,HC,ROPA LAVADO,Liquido,3000.0,25.65,30.4,20.9,1.6,15.91,False
2,201701,2017-01-01,20003,1063.45835,438,1067.81543,0,FOODS,ADEREZOS,Mayonesa,475.0,25.65,30.4,20.9,1.6,15.91,False
3,201701,2017-01-01,20004,555.91614,339,569.37394,0,FOODS,ADEREZOS,Mayonesa,240.0,25.65,30.4,20.9,1.6,15.91,False
4,201701,2017-01-01,20005,494.27011,249,494.60084,0,FOODS,ADEREZOS,Mayonesa,120.0,25.65,30.4,20.9,1.6,15.91,False


In [8]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31243 entries, 0 to 31242
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   periodo                    31243 non-null  int64  
 1   periodo_fecha              31243 non-null  object 
 2   product_id                 31243 non-null  int64  
 3   tn                         31243 non-null  float64
 4   cust_request_qty           31243 non-null  int64  
 5   cust_request_tn            31243 non-null  float64
 6   plan_precios_cuidados      31243 non-null  int64  
 7   cat1                       31243 non-null  object 
 8   cat2                       31243 non-null  object 
 9   cat3                       31243 non-null  object 
 10  sku_size                   31243 non-null  float64
 11  temp_media                 31243 non-null  float64
 12  temp_max_media             31243 non-null  float64
 13  temp_min_media             31243 non-null  flo

In [9]:
df_sellout.cat1.value_counts()

PC             16779
HC              7603
FOODS           6632
desconocida      229
Name: cat1, dtype: int64

In [10]:
# Selecciono las columnas iniciales
df_sellout = df_sellout[["periodo","product_id","tn","cat1"]]

Verificación de producto estrella --> ok

## Split train y test

In [11]:
df_train = df_sellout[df_sellout.periodo <= 201902]
df_test = df_sellout[df_sellout.periodo == 201904]

In [12]:
df_train

,periodo,product_id,tn,cat1
0,201701,20001,934.77222,HC
1,201701,20002,550.15707,HC
2,201701,20003,1063.45835,FOODS
3,201701,20004,555.91614,FOODS
4,201701,20005,494.27011,FOODS
...,...,...,...,...
21988,201902,21261,0.04656,PC
21989,201902,21262,0.05786,PC
21990,201902,21263,0.05927,PC
21991,201902,21264,0.01378,FOODS


In [13]:
print(len(df_test))
df_test

919


,periodo,product_id,tn,cat1
22888,201904,20001,1647.63848,HC
22889,201904,20002,1287.62346,HC
22890,201904,20003,565.33774,FOODS
22891,201904,20004,466.70901,FOODS
22892,201904,20005,624.99880,FOODS
...,...,...,...,...
23802,201904,21266,0.36178,PC
23803,201904,21267,0.12291,PC
23804,201904,21269,0.11351,PC
23805,201904,21271,0.00185,FOODS


In [14]:
product_id_nuevos = set(df_test.product_id).difference(set(df_train.product_id))
len(product_id_nuevos)

50

Hay 50 productos NUEVOS (están en 201904 y no antes)

Entonces vamos a predecir 919-50 => 869

In [15]:
df_test = df_test[~df_test.product_id.isin(product_id_nuevos)]
len(df_test)

869

In [16]:
# Como puede haber productos que no estén en algunos períodos (que se usan en modelos naif), calculo promedio para completar
df_completar = df_train[["product_id","tn"]].groupby("product_id").mean().reset_index()
df_completar

,product_id,tn
0,20001,1362.248849
1,20002,943.915757
2,20003,934.905062
3,20004,677.100396
4,20005,613.286967
...,...,...
1050,21295,0.006990
1051,21296,0.006510
1052,21297,0.005790
1053,21298,0.005730


## Predicción Estacional

In [17]:
df_pred_estac = df_train[df_train.periodo==201804]
df_pred_estac = df_pred_estac[["product_id","tn"]]
df_pred_estac.head()

,product_id,tn
12387,20001,1251.28462
12388,20002,999.20934
12389,20003,765.47838
12390,20004,611.51237
12391,20005,496.41774


In [18]:
# Left join de test con prediccion estacional
df_res_estac = pd.merge(df_test,df_pred_estac,on="product_id",how="left",suffixes=["_real","_estac"])

# Left join con prediccion promedio (para completar)
df_res_estac = pd.merge(df_res_estac,df_completar,on="product_id",how="left")
df_res_estac.rename(columns={'tn': 'tn_prom'}, inplace=True)

# Calculo toneladas predichas usando los valores anteriores
df_res_estac["tn_pred"]=np.where(df_res_estac.tn_estac.isnull(),df_res_estac.tn_prom,df_res_estac.tn_estac)

df_res_estac = df_res_estac[["product_id","cat1","tn_real","tn_pred"]]
df_res_estac.head()

,product_id,cat1,tn_real,tn_pred
0,20001,HC,1647.63848,1251.28462
1,20002,HC,1287.62346,999.20934
2,20003,FOODS,565.33774,765.47838
3,20004,FOODS,466.70901,611.51237
4,20005,FOODS,624.99880,496.41774


In [19]:
print_error_rate_total_y_cat(df_res_estac)

**Error Rate Total:** 34.9

**Error Rate por Categoría HC**: 33.44

**Error Rate por Categoría FOODS**: 32.1

**Error Rate por Categoría PC**: 42.89

## Predicción Menos 2

In [20]:
# Naif -2
df_pred_menos2 = df_train[df_train.periodo==201902]
df_pred_menos2 = df_pred_menos2[["product_id","tn"]]
df_pred_menos2.head()

,product_id,tn
21099,20001,1259.09363
21100,20002,1043.01349
21101,20003,758.32657
21102,20004,441.70332
21103,20005,409.89950


In [21]:
# Left join de test con prediccion menos 2
df_res_menos2 = pd.merge(df_test,df_pred_menos2,on="product_id",how="left",suffixes=["_real","_menos2"])

# Left join con prediccion promedio (para completar)
df_res_menos2 = pd.merge(df_res_menos2,df_completar,on="product_id",how="left")
df_res_menos2.rename(columns={'tn': 'tn_prom'}, inplace=True)

# Calculo toneladas predichas usando los valores anteriores
df_res_menos2["tn_pred"]=np.where(df_res_menos2.tn_menos2.isnull(),df_res_menos2.tn_prom,df_res_menos2.tn_menos2)

df_res_menos2 = df_res_menos2[["product_id","cat1","tn_real","tn_pred"]]
df_res_menos2.head()

,product_id,cat1,tn_real,tn_pred
0,20001,HC,1647.63848,1259.09363
1,20002,HC,1287.62346,1043.01349
2,20003,FOODS,565.33774,758.32657
3,20004,FOODS,466.70901,441.70332
4,20005,FOODS,624.99880,409.89950


In [22]:
print_error_rate_total_y_cat(df_res_menos2)

**Error Rate Total:** 30.0

**Error Rate por Categoría HC**: 28.98

**Error Rate por Categoría FOODS**: 27.45

**Error Rate por Categoría PC**: 36.2